In [12]:
from datetime import datetime
from sqlalchemy import create_engine

engine = create_engine(
    "sqlite+pysqlite:///user.db", 
    echo=True, 
    future=True
)

from sqlalchemy import (
    Column, 
    Integer, 
    String,
    DateTime,
    ForeignKey,
)
from sqlalchemy.orm import declarative_base, relationship

Base = declarative_base()

class User(Base):
    __tablename__ = "user"
    
    id = Column(Integer, primary_key=True)
    first_name = Column(String(50), nullable=False)
    last_name = Column(String(50), nullable=False)
    age = Column(Integer, nullable=False)
    
    def __str__(self):
        return f"Customer <id:{self.id}, first_name:{self.first_name}, last_name:{self.last_name}, age:{self.age}>"
    
Base.metadata.create_all(engine)

2021-09-04 13:33:10,415 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-04 13:33:10,417 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2021-09-04 13:33:10,418 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-04 13:33:10,422 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user")
2021-09-04 13:33:10,423 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-04 13:33:10,426 INFO sqlalchemy.engine.Engine 
CREATE TABLE user (
	id INTEGER NOT NULL, 
	first_name VARCHAR(50) NOT NULL, 
	last_name VARCHAR(50) NOT NULL, 
	age INTEGER NOT NULL, 
	PRIMARY KEY (id)
)


2021-09-04 13:33:10,429 INFO sqlalchemy.engine.Engine [no key 0.00297s] ()
2021-09-04 13:33:10,442 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
# Создадим базовый класс для сессии
from sqlalchemy.orm import sessionmaker


Session = sessionmaker(engine)
# создаем сессию
with Session() as session:

    users = [User(first_name="Sergei", last_name="Dmitriev", age="24"),
         User(first_name="Alex", last_name="Vyazhevich", age="15"),
         User(first_name="Tom", last_name="Redl", age="19"),
         User(first_name="Sergei", last_name="Petrov", age="17"),
         User(first_name="Petr", last_name="Kovalsky", age="22")]

    session.add_all(users)
    session.commit()


2021-09-04 13:33:14,092 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-04 13:33:14,096 INFO sqlalchemy.engine.Engine INSERT INTO user (first_name, last_name, age) VALUES (?, ?, ?)
2021-09-04 13:33:14,099 INFO sqlalchemy.engine.Engine [generated in 0.00265s] ('Sergei', 'Dmitriev', '24')
2021-09-04 13:33:14,104 INFO sqlalchemy.engine.Engine INSERT INTO user (first_name, last_name, age) VALUES (?, ?, ?)
2021-09-04 13:33:14,106 INFO sqlalchemy.engine.Engine [cached since 0.009286s ago] ('Alex', 'Vyazhevich', '15')
2021-09-04 13:33:14,108 INFO sqlalchemy.engine.Engine INSERT INTO user (first_name, last_name, age) VALUES (?, ?, ?)
2021-09-04 13:33:14,113 INFO sqlalchemy.engine.Engine [cached since 0.01638s ago] ('Tom', 'Redl', '19')
2021-09-04 13:33:14,123 INFO sqlalchemy.engine.Engine INSERT INTO user (first_name, last_name, age) VALUES (?, ?, ?)
2021-09-04 13:33:14,130 INFO sqlalchemy.engine.Engine [cached since 0.03377s ago] ('Sergei', 'Petrov', '17')
2021-09-04 13:33:14,133 INFO 

In [14]:
with Session() as session:
    Sergei = session.query(User).filter(
        User.first_name == "Sergei"
    )
    for a in Sergei.all():
        print(a)

2021-09-04 13:33:18,214 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-04 13:33:18,223 INFO sqlalchemy.engine.Engine SELECT user.id AS user_id, user.first_name AS user_first_name, user.last_name AS user_last_name, user.age AS user_age 
FROM user 
WHERE user.first_name = ?
2021-09-04 13:33:18,225 INFO sqlalchemy.engine.Engine [generated in 0.00224s] ('Sergei',)
Customer <id:1, first_name:Sergei, last_name:Dmitriev, age:24>
Customer <id:4, first_name:Sergei, last_name:Petrov, age:17>
2021-09-04 13:33:18,232 INFO sqlalchemy.engine.Engine ROLLBACK


In [15]:
with Session() as session:
    Sergei = session.query(User).filter(
        User.age < 25
    )
    for a in Sergei.all():
        print(a)

2021-09-04 13:34:04,480 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-04 13:34:04,489 INFO sqlalchemy.engine.Engine SELECT user.id AS user_id, user.first_name AS user_first_name, user.last_name AS user_last_name, user.age AS user_age 
FROM user 
WHERE user.age < ?
2021-09-04 13:34:04,491 INFO sqlalchemy.engine.Engine [generated in 0.00241s] (25,)
Customer <id:1, first_name:Sergei, last_name:Dmitriev, age:24>
Customer <id:2, first_name:Alex, last_name:Vyazhevich, age:15>
Customer <id:3, first_name:Tom, last_name:Redl, age:19>
Customer <id:4, first_name:Sergei, last_name:Petrov, age:17>
Customer <id:5, first_name:Petr, last_name:Kovalsky, age:22>
2021-09-04 13:34:04,497 INFO sqlalchemy.engine.Engine ROLLBACK


In [16]:
with Session() as session:
    Sergei = session.query(User).filter(
        User.age.between(18, 25)
    )
    for a in Sergei.all():
        print(a)

2021-09-04 13:36:00,595 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-04 13:36:00,599 INFO sqlalchemy.engine.Engine SELECT user.id AS user_id, user.first_name AS user_first_name, user.last_name AS user_last_name, user.age AS user_age 
FROM user 
WHERE user.age BETWEEN ? AND ?
2021-09-04 13:36:00,601 INFO sqlalchemy.engine.Engine [generated in 0.00218s] (18, 25)
Customer <id:1, first_name:Sergei, last_name:Dmitriev, age:24>
Customer <id:3, first_name:Tom, last_name:Redl, age:19>
Customer <id:5, first_name:Petr, last_name:Kovalsky, age:22>
2021-09-04 13:36:00,607 INFO sqlalchemy.engine.Engine ROLLBACK
